In [80]:
# Load libraries
library(tidyverse)


In [2]:
# This cell downloads the original data from data.bs.ch and saves it in the raw folder
# It has been downloaded on 2024-03-01

# download_link <- "https://data.bs.ch/api/explore/v2.1/catalog/datasets/100138/exports/csv?lang=de&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
# download.file(download_link, destfile = "/raw/100138.csv")


In [81]:
# Load original data
original_data <- read_delim("raw/100138.csv", delim = ";")


Rows: 1664159 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (14): Wanderungstyp, Wochentag, Staatsangehörigkeit, Geschlecht, Von Ko...
dbl   (7): Jahr, Monat, Kalenderwoche, Tag-Nr., Alter, Aufenthaltsdauer in J...
date  (2): Datum, Startdatum Woche

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [82]:
# Peek into the original data
head(original_data)


Wanderungstyp,Datum,Jahr,Monat,Kalenderwoche,Startdatum Woche,Tag-Nr.,Wochentag,Staatsangehörigkeit,Geschlecht,⋯,Von Land,Von Kanton,Von Gemeinde,Von Wohnviertel,Nach Kontinent,Nach Land,Nach Kanton,Nach Gemeinde,Nach Wohnviertel,Anzahl
<chr>,<date>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Zuzug,2003-08-08,2003,8,32,2003-08-04,220,Fr,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Riehen,Riehen,1
Zuzug,2003-08-08,2003,8,32,2003-08-04,220,Fr,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Riehen,Riehen,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Ausländer,M,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Breite,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Gundeldingen,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Schweizer,M,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,St. Alban,2
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Schweizer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Iselin,1


In [83]:
# Save new dataframe with less columns
data_reduced <- select(
  original_data,
  -"Datum",
  -"Kalenderwoche",
  -"Startdatum Woche",
  -"Tag-Nr.",
  -"Wochentag",
  -"Aufenthaltsdauer in Jahren",
  -"Von Gemeinde",
  -"Nach Gemeinde"
)


In [84]:
# Remove spaces from column names
colnames(data_reduced) <- make.names(colnames(data_reduced))


In [85]:
# Rename column names
data_reduced <- rename(data_reduced,
  Staatsangehoerigkeit = Staatsangehörigkeit,
  VonKontinent = Von.Kontinent,
  VonLand = Von.Land,
  VonKanton = Von.Kanton,
  VonWohnviertel = Von.Wohnviertel,
  NachKontinent = Nach.Kontinent,
  NachLand = Nach.Land,
  NachKanton = Nach.Kanton,
  NachWohnviertel = Nach.Wohnviertel
)


In [86]:
# Filter out all migrations within Basel
data_reduced <- filter(data_reduced, Wanderungstyp != "Umzug")

In [87]:
# Replace "Unbekannt" or "unbekannt" or "--" with NA
data_reduced <- mutate_all(data_reduced, ~ ifelse(. %in% c("Unbekannt", "unbekannt", "--"), NA, .))

In [89]:
# Sort data
data_reduced <- arrange(data_reduced, Jahr, Monat)


In [90]:
# Repeat each row n times if Anzahl is > 1
# For the final visualization, we want to have an individual data point for each migration

# Create a vector of row indices to repeat
row_indices <- rep(seq_len(nrow(data_reduced)), ifelse(data_reduced$Anzahl > 1, data_reduced$Anzahl - 1, 0))

# Create the expanded dataframe by indexing the original dataframe
expanded_data <- data_reduced[row_indices, ]


In [91]:
# Bind the copied rows to the  data
data_modified <- rbind(data_reduced, expanded_data)


In [92]:
# Re-sort the data
data_modified <- arrange(data_modified, Jahr, Monat, Alter)


In [93]:
# Remove the Anzahl column, it's no longer needed
data_modified <- select(data_modified, -"Anzahl")


In [95]:
# The geojson of Switzerland used in the frontend doesn't have
# canton abbreviations, so we'll map our data here accordingly

cantons <- list(
  AG = "Aargau",
  AI = "Appenzell Innerrhoden",
  AR = "Appenzell Ausserrhoden",
  BE = "Bern",
  BL = "Basel-Landschaft",
  BS = "Basel-Stadt",
  FR = "Fribourg",
  GE = "Genève",
  GL = "Glarus",
  GR = "Graubünden",
  JU = "Jura",
  LU = "Luzern",
  NE = "Neuchâtel",
  NW = "Nidwalden",
  OW = "Obwalden",
  SG = "St. Gallen",
  SH = "Schaffhausen",
  SO = "Solothurn",
  SZ = "Schwyz",
  TG = "Thurgau",
  TI = "Ticino",
  UR = "Uri",
  VD = "Vaud",
  VS = "Valais",
  ZG = "Zug",
  ZH = "Zürich"
)


In [96]:
# The geojson of the world used in the frontend doesn't
# have German country names, so we'll map our data here accordingly.

# Also, the world map's scale doesn't allow to plot every small country/state/island
# like Andorra, Singapore, etc. These are mapped to its surrounding areas,
# even though that might not be politically correct.
# I apologize if this offends anyone. It's about plotting the very
# rough location on the map, not about political belongings.

countries <- list(
  "Aegypten" = "Egypt",
  "Aethiopien" = "Ethiopia",
  "Afghanistan" = "Afghanistan",
  "Albanien" = "Albania",
  "Algerien" = "Algeria",
  "Andorra" = "Spain",
  "Angola" = "Angola",
  "Antarktis & Antarktische Ins." = "Unknown",
  "Argentinien" = "Argentina",
  "Armenien" = "Armenia",
  "Aruba" = "Venezuela",
  "Aserbaidschan" = "Azerbaijan",
  "Australien" = "Australia",
  "Bahamas" = "Bahamas",
  "Bahrain" = "Saudi Arabia",
  "Bangladesch" = "Bangladesh",
  "Barbados" = "Venezuela",
  "Belgien" = "Belgium",
  "Belize" = "Belize",
  "Benin" = "Benin",
  "Bermuda Inseln" = "Puerto Rico",
  "Bhutan" = "Bhutan",
  "Bolivien" = "Bolivia",
  "Bosnien-Herzegowina" = "Bosnia and Herz.",
  "Botswana" = "Botswana",
  "Brasilien" = "Brazil",
  "Brunei" = "Malaysia",
  "Bulgarien" = "Bulgaria",
  "Bundesrep. Jugoslawien" = "Serbia",
  "Burkina Faso" = "Burkina Faso",
  "Burundi" = "Burundi",
  "Chile" = "Chile",
  "China" = "China",
  "Costa Rica" = "Costa Rica",
  "Dänemark" = "Denmark",
  "Deutschland" = "Germany",
  "Dominica" = "Puerto Rico",
  "Dominikanische Republik" = "Dominican Rep.",
  "Ecuador" = "Ecuador",
  "El Salvador" = "El Salvador",
  "Elfenbeinküste" = "Côte d'Ivoire",
  "Eritrea" = "Eritrea",
  "Estland" = "Estonia",
  "Fidschi" = "Fiji",
  "Finnland" = "Finland",
  "Frankreich" = "France",
  "Französisch Polynesien" = "Fiji",
  "Gabun" = "Gabon",
  "Gambia" = "Gambia",
  "Georgien" = "Georgia",
  "Ghana" = "Ghana",
  "Grenada" = "Venezuela",
  "Griechenland" = "Greece",
  "Grossbritannien" = "United Kingdom",
  "Guadeloupe" = "Puerto Rico",
  "Guatemala" = "Guatemala",
  "Guernsey" = "France",
  "Guinea-Bissau" = "Guinea-Bissau",
  "Guinea" = "Guinea",
  "Guyana (Französisch)" = "Suriname",
  "Haiti" = "Haiti",
  "Honduras" = "Honduras",
  "Hong Kong" = "China",
  "Indien" = "India",
  "Indonesien" = "Indonesia",
  "Irak" = "Iraq",
  "Iran" = "Iran",
  "Irland" = "Ireland",
  "Island" = "Iceland",
  "Israel" = "Israel",
  "Italien" = "Italy",
  "Jamaika" = "Jamaica",
  "Japan" = "Japan",
  "Jemen (Arabische Republik)" = "Yemen",
  "Jordanien" = "Jordan",
  "Jungferninseln (UK)" = "Puerto Rico",
  "Kaimaninseln" = "Cuba",
  "Kambodscha" = "Cambodia",
  "Kamerun" = "Cameroon",
  "Kanada" = "Canada",
  "Kanalinseln" = "France",
  "Kanarische Inseln" = "Morocco",
  "Kap Verde" = "Senegal",
  "Kasachstan" = "Kazakhstan",
  "Katar" = "Qatar",
  "Kenia" = "Kenya",
  "Kirgistan" = "Kyrgyzstan",
  "Kolumbien" = "Colombia",
  "Kongo (Brazzaville)" = "Dem. Rep. Congo",
  "Kongo (Kinshasa)" = "Dem. Rep. Congo",
  "Korea (Nord-)" = "North Korea",
  "Korea (Süd-)" = "South Korea",
  "Kosovo" = "Kosovo",
  "Kroatien" = "Croatia",
  "Kuba" = "Cuba",
  "Kuwait" = "Kuwait",
  "Laos" = "Laos",
  "Lesotho" = "Lesotho",
  "Lettland" = "Latvia",
  "Libanon" = "Lebanon",
  "Liberia" = "Liberia",
  "Libyen" = "Libya",
  "Liechtenstein" = "Austria",
  "Litauen" = "Lithuania",
  "Luxemburg" = "Luxembourg",
  "Madagaskar" = "Madagascar",
  "Malawi" = "Malawi",
  "Malaysia" = "Malaysia",
  "Malediven" = "Sri Lanka",
  "Mali" = "Mali",
  "Malta" = "Italy",
  "Marokko" = "Morocco",
  "Martinique" = "Puerto Rico",
  "Mauretanien" = "Mauritania",
  "Mauritius" = "Madagascar",
  "Mayotte" = "Madagascar",
  "Mexiko" = "Mexico",
  "Moldawien" = "Moldova",
  "Monaco" = "France",
  "Mongolei" = "Mongolia",
  "Montenegro" = "Montenegro",
  "Mosambik" = "Mozambique",
  "Myanmar" = "Myanmar",
  "Namibia" = "Namibia",
  "Nepal" = "Nepal",
  "Neuseeland" = "New Zealand",
  "Nicaragua" = "Nicaragua",
  "Niederlande" = "Netherlands",
  "Niederländische Antillen" = "Dominican Rep.",
  "Niger" = "Niger",
  "Nigeria" = "Nigeria",
  "Nordmazedonien" = "North Macedonia",
  "Norwegen" = "Norway",
  "Oesterreich" = "Austria",
  "Oman" = "Oman",
  "Pakistan" = "Pakistan",
  "Palästina" = "Palestine",
  "Panama" = "Panama",
  "Papua-Neuguinea" = "Papua New Guinea",
  "Paraguay" = "Paraguay",
  "Peru" = "Peru",
  "Philippinen" = "Philippines",
  "Polen" = "Poland",
  "Portugal" = "Portugal",
  "Provinz Wojwodina" = "Serbia",
  "Reunion" = "Madagascar",
  "Ruanda" = "Rwanda",
  "Rumänien" = "Romania",
  "Russland" = "Russia",
  "Sambia" = "Zambia",
  "Samoa" = "Fiji",
  "San Marino" = "Italy",
  "Saudi-Arabien" = "Saudi Arabia",
  "Schweden" = "Sweden",
  "Schweiz" = "Switzerland",
  "Senegal" = "Senegal",
  "Serbien" = "Serbia",
  "Seychellen" = "Madagascar",
  "Sierra Leone" = "Sierra Leone",
  "Simbabwe" = "Zimbabwe",
  "Singapur" = "Malaysia",
  "Slowakische Republik" = "Slovakia",
  "Slowenien" = "Slovenia",
  "Somalia" = "Somalia",
  "Spanien" = "Spain",
  "Sri Lanka" = "Sri Lanka",
  "St.Kitts und Nevis" = "Puerto Rico",
  "St.Lucia" = "Puerto Rico",
  "St.Vincent und die Grenadinen" = "Puerto Rico",
  "Südafrika" = "South Africa",
  "Sudan" = "Sudan",
  "Südsudan" = "Sudan",
  "Surinam" = "Suriname",
  "Syrien" = "Syria",
  "Tadschikistan" = "Tajikistan",
  "Taiwan (Chinesisches Taipei)" = "Taiwan",
  "Tansania" = "Tanzania",
  "Thailand" = "Thailand",
  "Tibet" = "Bhutan",
  "Togo" = "Togo",
  "Tonga" = "Fiji",
  "Trinidad und Tobago" = "Trinidad and Tobago",
  "Tschad" = "Chad",
  "Tschechische Republik" = "Czechia",
  "Tunesien" = "Tunisia",
  "Türkei" = "Turkey",
  "Turkmenistan" = "Turkmenistan",
  "Uganda" = "Uganda",
  "Ukraine" = "Ukraine",
  "Ungarn" = "Hungary",
  "Uruguay" = "Uruguay",
  "Usbekistan" = "Uzbekistan",
  "Vanuatu" = "Vanuatu",
  "Venezuela" = "Venezuela",
  "Vereinigte Arabische Emirate" = "United Arab Emirates",
  "Vereinigte Staaten von Amerika" = "United States of America",
  "Vietnam" = "Vietnam",
  "Weissrussland" = "Belarus",
  "Zentralserbien" = "Serbia",
  "Zypern" = "Cyprus"
)


In [158]:
# We are going to add some data to each row so we don't have to do it in the frontend later
# We want to keep the starting map and region as well as the end map and region of each row (e.g. "Basel"/"St. Johann" or "World"/"USA", etc.)

data_enhanced <- add_column(data_modified,
  StartKarte = NA_character_,
  StartRegion = NA_character_,
  EndKarte = NA_character_,
  EndRegion = NA_character_
)

In [159]:
# Set the starting maps
data_enhanced <- mutate(data_enhanced,
  StartKarte = case_when(
    Wanderungstyp == "Wegzug" ~ "basel", # "Basel" for all emigrants
    VonKontinent == "Schweiz" ~ "switzerland",
    VonKontinent == "Europa (ohne Schweiz)" &
      (VonLand != "Russland" & VonLand != "Türkei" | is.na(VonLand)) ~
      "europe",
    VonLand == "Russland" | VonLand == "Türkei" ~ "world",
    is.na(VonKontinent) ~ NA,
    TRUE ~ "world"
  )
)


In [160]:
# Set the starting region
data_enhanced <- mutate(data_enhanced,
  StartRegion = ifelse(
    StartKarte == "basel",
    VonWohnviertel, as.character(ifelse(
      StartKarte == "switzerland",
      ifelse(is.na(VonKanton), NA, cantons[VonKanton]),
      ifelse(is.na(VonLand), NA, countries[VonLand])
    ))
  )
)


In [161]:
# Set the ending maps
data_enhanced <- mutate(data_enhanced,
  EndKarte = case_when(
    Wanderungstyp == "Zuzug" ~ "basel", # "Basel" for all imigrants
    NachKontinent == "Schweiz" ~ "switzerland",
    NachKontinent == "Europa (ohne Schweiz)" &
      (NachLand != "Russland" & NachLand != "Türkei" | is.na(NachLand)) ~
      "europe",
    NachLand == "Russland" | NachLand == "Türkei" ~ "world",
    is.na(NachKontinent) ~ NA,
    TRUE ~ "world"
  )
)

In [162]:
# Set the ending region
data_enhanced <- mutate(data_enhanced,
  EndRegion = ifelse(
    EndKarte == "basel",
    NachWohnviertel, as.character(ifelse(
      EndKarte == "switzerland",
      ifelse(is.na(NachKanton), NA, cantons[NachKanton]),
      ifelse(is.na(NachLand), NA, countries[NachLand])
    ))
  )
)

In [164]:
# Save project data as CSV
write_csv(data_enhanced, "tidy/migration.csv")


In [110]:
# In order to host the data on GitHub, we need to divide the migration data into smaller chunks.
# GitHub has a file size limit of 50MB.
# However, we are going to create much more chunks - one per month!!
# On the website, users will be able to select a starting year and month.
# The site will load only the needed data.
# Once the animation is done, the subsequent data will be loaded.


In [165]:
# Read final data
final_data <- read_delim("tidy/migration.csv", delim = ",")


Rows: 950150 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (15): Wanderungstyp, Staatsangehoerigkeit, Geschlecht, VonKontinent, Von...
dbl  (3): Jahr, Monat, Alter

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [166]:
chunks <- final_data %>%
  mutate(JahrMonat = paste(Jahr, Monat, sep = "-"), .before = 1) %>%
  group_by(JahrMonat) %>%
  group_split()


In [167]:
# Save each chunk as a separate csv file
output_dir <- "tidy/chunks/"
for (i in seq_along(chunks)) {
  write_csv(chunks[[i]], paste0(output_dir, chunks[[i]][1, "JahrMonat"], ".csv"))
}


In [168]:
# Copy files to the web apps directory
source_directory <- "tidy/chunks"
target_directory <- "../basel-migration/public/data/migration"

# Get list of file names in the source directory
file_list <- list.files(source_directory, full.names = TRUE)

# Copy files to the target directory
file.copy(from = file_list, to = target_directory, overwrite = TRUE)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[136] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[151] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[166] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[181] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[196] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[211] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[226] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[241] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[256] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[271] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[286] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[301] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[316] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[331] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[346] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[361] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[376] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[391] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[406] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[421] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[436] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[451] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE